In [1]:
import pyfesom2 as pf
import xarray as xr
import numpy as np

osgeo is not installed, conversion to Geo formats like Geotiff (fesom2GeoFormat) will not work.


In [ ]:
mesh_path = 'path_to_your_mesh'
data_path = 'path_to_your_model_output'

#which year?
year = 2000
#chunk size in time?
time_chunk = 1
#chunk size in space?
nod2_chunk = 10000000
#which depth?
depth = 100

In [2]:
#fesom.mesh.diag.nc contains the dx/dy operators (basically just vectors of the same size as u/v)
diag = xr.open_dataset((mesh_path+'fesom.mesh.diag.nc'))  
ddx = diag['gradient_sca_x']
ddy = diag['gradient_sca_y']


In [ ]:
###### for old FESOM2 versions 
elems = (diag['elements']-1).T.astype('int')  #element indices are saved in Fortran format, starting from 1 instead of 0
elems = elems[:, [0, 2, 1]] #element indices are in the wrong order from Fortran!!!!!!

In [ ]:
###### for new FESOM2 versions
# elems = (diag['elements']-1).astype('int')

In [2]:
iz = np.argmin(np.abs(diag.nz1.values+depth)) #find the closest level in the model, depth in diag.nz1 is negative, that is why I use +depth

unod = xr.open_dataset((data_path+'unod.fesom.' + str(year) + '.nc'),chunks={'time':time_chunk,'nod2':nod2_chunk})['unod'].astype('float32').isel(nz1=iz)
vnod = xr.open_dataset((data_path+'vnod.fesom.' + str(year) + '.nc'),chunks={'time':time_chunk,'nod2':nod2_chunk})['vnod'].astype('float32').isel(nz1=iz)

day_index = dd # pick a day to work on for now. I want to write a function that can be thrown on a full array, automatically calculating every day in parallel, but I am not there yet

v=vnod[dd,:].compute()
u=unod[dd,:].compute()

dv_dx = (ddx * v[elem]).sum(dim='n3') 
du_dy = (ddy * u[elem]).sum(dim='n3')
dv_dy = (ddy * v[elem]).sum(dim='n3') 
du_dx = (ddx * u[elem]).sum(dim='n3')

curl = dv_dx - du_dy
curl = pf.tonodes(curl,mesh)

SyntaxError: invalid syntax (194925481.py, line 1)

In [ ]:
# or as a function:
def curl2d(u,v,mesh,diag):
#    u,v = pf.vec_rotate_r2g(50, 15, -90, model_lons, model_lats, u,v, flag=1) dont rotate!! diag['gradient_sca_x'] is wrt to rotated field
    dv_dx = ((diag['gradient_sca_x']) * v[mesh.elem]).sum(axis=1)
    du_dy = ((diag['gradient_sca_y']) * u[mesh.elem]).sum(axis=1)
    curl = dv_dx - du_dy
    curl = pf.tonodes(curl,mesh)
    return curl

#might be better to pass ddx/ddy because diag contains a LOT more than just those and it might lead to some overhead